In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sqlite3

In [3]:
con = sqlite3.connect('database.sqlite') 

In [4]:
data = pd.read_sql_query("""SELECT * FROM Reviews""", con)

In [5]:
data.shape

(568454, 10)

In [6]:
data = data[data['Score'] != 3]

In [7]:
def posi_negi(rat):
    if rat < 3:
        return 'negative'
    return 'positive'

In [8]:
actualScore = np.array(data['Score'])

In [9]:
actualScore = list(map(posi_negi,actualScore))

In [10]:
len(actualScore)

525814

In [11]:
data['Score'] = actualScore

In [12]:
print(data.shape)
data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [13]:
data['Score'].value_counts()

positive    443777
negative     82037
Name: Score, dtype: int64

# Exploratory Data Analysis & Pre-Processing

Handling the duplicates

In [14]:
data[data.duplicated({'Text','Summary','Time','ProfileName','UserId'})].head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
29,30,B0001PB9FY,A3HDKO7OW0QNK4,Canadian Fan,1,1,positive,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...
574,575,B000G6RYNE,A3PJZ8TU8FDQ1K,Jared Castle,2,2,positive,1231718400,"One bite and you'll become a ""chippoisseur""","I'm addicted to salty and tangy flavors, so wh..."
2309,2310,B0001VWE0M,AQM74O8Z4FMS0,Sunshine,0,0,negative,1127606400,Below standard,Too much of the white pith on this orange peel...
2323,2324,B0001VWE0C,AQM74O8Z4FMS0,Sunshine,0,0,negative,1127606400,Below standard,Too much of the white pith on this orange peel...
2946,2947,B0002TJAZK,A2ISKAWUPGGOLZ,M. S. Handley,0,1,negative,1310774400,Kitty Junk Food,We have five cats - one an elderly cat of 15 y...


In [15]:
data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [16]:
sorted_data = data.sort_values(['ProductId'])

In [17]:
final_data = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})

In [18]:
final_data.shape

(364173, 10)

Removing cases where HelpfullnessNumeraot > HelpfullnessDenominator

In [19]:
final_data[final_data['HelpfulnessNumerator'] > final_data['HelpfulnessDenominator']]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,positive,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,positive,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [20]:
final_data = final_data[final_data['HelpfulnessNumerator'] <= final_data['HelpfulnessDenominator']]

In [21]:
final_data.shape

(364171, 10)

In [22]:
final_data['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

Checking for html tags

In [34]:
import re
i=0
for sentence in final_data['Text'].values:
    if (len(re.findall('<.*?>', sentence))):
        print("index : ",i)
        print(sentence)
        break;
    i = i+1

index :  6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [71]:
# html tags cleaner
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
# punctuation cleaner
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prabh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [44]:
# stopwords
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [47]:
stop = set(stopwords.words('english'))

In [48]:
sno = nltk.stem.SnowballStemmer('english')

In [49]:
print(stop)

{'themselves', 'been', 'no', 'these', 'those', 'have', 'a', 'through', "mightn't", "she's", "wasn't", 'over', "you'd", 'just', 'or', 'i', 'then', 'shouldn', 'not', 'are', 'and', 'into', 'they', 'll', 'if', 'so', 'who', 'my', "doesn't", 'too', 'them', 'were', 'both', 'm', 'most', 'such', 'which', 'than', 'few', 'should', 'be', 'isn', 'hasn', 'do', 'of', 'where', 'yourself', 'up', 'yourselves', 'as', "isn't", 'we', 'above', 'haven', 'd', 'aren', 'your', 'is', 'down', "couldn't", 'very', 're', 'there', 'you', "won't", 'this', 'mightn', 'here', 'yours', 'because', 'don', 'wasn', 'against', "you've", 'wouldn', 'hadn', 's', 'own', 'she', 'off', 'him', 't', 'out', 'about', "you're", 'at', 'mustn', 'all', 'some', 'has', "didn't", "that'll", 'what', 'its', 'to', 'now', 'doesn', 'herself', 'the', "weren't", 'more', 'by', 'ours', 'only', 'needn', 'was', 'that', 'in', 've', 'but', 'same', 'couldn', 'weren', 'won', 'did', 'between', "haven't", 'am', "aren't", 'ma', 'he', "it's", 'whom', 'ain', 'wil

In [52]:
stop.remove('not')

In [53]:
stop.remove('no')

In [74]:
# Computation

i=0
posi_words = []
negi_words = []
str1=' '
final_string=[]
for sent in final_data['Text'].values:
    filtered_sent = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_word in cleanpunc(w).split():
            if (cleaned_word.isalpha() and len(cleaned_word)>2):
                if(cleaned_word.lower() not in stop):
                    s = (sno.stem(cleaned_word.lower())).encode('utf8')
                    filtered_sent.append(s)
                    if(final_data['Score'].values[i] == 'positive'):
                        posi_words.append(s)
                    elif(final_data['Score'].values[i] == 'negative'):
                        negi_words.append(s)
                else:
                    continue
            else:
                continue
    str1 = b" ".join(filtered_sent)
    final_string.append(str1)
    i+=1

In [77]:
len(final_string)

364171

In [78]:
final_string[0]

b'witti littl book make son laugh loud recit car drive along alway sing refrain learn whale india droop love new word book introduc silli classic book will bet son still abl recit memori colleg'

In [79]:
final_data['CleanedText'] = final_string

In [81]:
final_data['CleanedText'] = final_data['CleanedText'].str.decode('utf8')

In [83]:
final_data.to_csv('final_data.csv',index=False)

In [90]:
posi_dist = nltk.FreqDist(posi_words)

In [91]:
negi_dist = nltk.FreqDist(negi_words)

In [94]:
# for future bow reference